In [1]:
# Get ERT times
import pandas as pd
ert_surveys_file = "survey_times_ERT.csv"
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
ert_surveys = pd.read_csv(ert_surveys_file, parse_dates=["time_local_start"], date_parser=dateparse)
ert_surveys["time_local_start"] = ert_surveys["time_local_start"].dt.tz_localize("America/Edmonton", ambiguous="infer")
ert_surveys["time_utc_start"] = ert_surveys["time_local_start"].dt.tz_convert(None)
ert_surveys["time_utc_end"] = ert_surveys["time_utc_start"] + pd.Timedelta(30, unit="m")
ert_surveys["time_utc_end"] = pd.to_datetime(ert_surveys["time_utc_end"])
ert_surveys["time_utc_start"] = pd.to_datetime(ert_surveys["time_utc_start"])

/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [143]:
# FUNCTIONS

def plot_stack_singlechannels(streams, stack):
    
    ids = [tr.get_id() for tr in streams[0]]
    n = len(ids)
    ncols = 3
    nrows = math.ceil(n/ncols)
    print("n = %d, nrows = %d, ncols = %d" % (n, nrows, ncols))
    
    fig, axs = plt.subplots(nrows, ncols, sharex=True, figsize=(15, 5*nrows))    
    for tr_id, ax in zip(ids, axs.flatten()):
        
        # Plot each event in background
        for st in streams:
            tr = st.select(id=tr_id)
            if not len(tr):
                continue
            else:
                tr = tr[0]             
            t = tr.times("utcdatetime") - tr.stats.starttime
            ax.plot(t, tr.data/tr.max(), color="gray", marker=None, linestyle="-")        
        # Plot stack
        tr = stack.select(id=tr_id)[0]            
        ax.plot(t, tr.data/tr.max(), color="red", marker=None, linestyle="-")
        ax.set_title("%s" % (tr_id))
    plt.show()
    plt.close()

def get_full_template(template):
    st = template.st
    tmin = min([tr.stats.starttime for tr in st])
    wf_len_s = int(8.0/ndt)*ndt
    prepick = 4.0
    pattern = os.path.join(WF_DIR_ROOT_500Hz, tmin.strftime("%Y%m%d"), "*DP*")
    detst = read(pattern, starttime=tmin - prepick, endtime=tmin - prepick + wf_len_s)
    pattern_hawk = os.path.join(WF_DIR_ROOT_HAWK, "*", "*", "*DP*.D.%s*" % tmin.strftime("%Y%m%d"))
    if glob(pattern_hawk):
        detst += read(pattern_hawk, starttime=tmin - prepick, endtime=tmin - prepick + wf_len_s)
    detst.detrend("demean")
    return detst

def get_stack(family):
    
    # Get waveforms
    streams = []
    wf_len_s = int(6.0/ndt)*ndt
    prepick = 2.0
    wf_len = int(wf_len_s/ndt) + 1
    for d in family.detections:        
        #print(d.detect_time)
        pattern = os.path.join(WF_DIR_ROOT, "*", "*..DP*%s*2020*" % (d.detect_time.strftime("%Y-%m-%d_%H")))                                      
        detst = read(pattern, starttime=d.detect_time - prepick, endtime=d.detect_time - prepick + wf_len_s)
        detst.resample(SAMPLING_RATE)
        detst.detrend("demean")
        # Check if traces have same length
        lengths = set([len(tr) for tr in detst])
        if len(lengths) > 1:
            print("Traces don't have same lengths. Fixing")
            tmin = min([tr.stats.starttime for tr in detst])
            tmax = tmin + wf_len
            detst.trim(starttime=tmin, endtime=tmax, fill_value=0, pad=True)
            print(detst)
        
        streams.append(detst)

    lengths = [len(st[0].data) for st in streams]    
    igood = np.argwhere([len(st[0].data) == wf_len for st in streams]).flatten()
    streams = [streams[i] for i in igood]    
    stack = linstack(streams)
    #plot_stack_3c(streams, stack)
#     plot_stack_singlechannels(streams, stack)
    return stack


def plot_stack_3c(streams, stack):
    stations = list(set([tr.stats.station for tr in stack]))
    nsta = len(stations)    
    fig, axs = plt.subplots(nsta, 3, sharex=True, figsize=(15, 5*nsta))
    for i, station in enumerate(stations):
        available_chans = [tr.stats.channel for tr in streams[0].select(station=station)]
        for ich, channel in enumerate(available_chans): #["DPN", "DPE", "DPZ"]):            
            ax = axs[i][ich]
            # Plot each event in background
            for st in streams:
                tr = st.select(station=station, channel=channel)[0]                
                t = tr.times("utcdatetime") - tr.stats.starttime
                ax.plot(t, tr.data/tr.max(), color="gray", marker=None, linestyle="-")
            # Plot stack
            tr = stack.select(station=station, channel=channel)[0]            
            ax.plot(t, tr.data/tr.max(), color="red", marker=None, linestyle="-")
            ax.set_ylabel("%s.%s" % (tr.stats.station, tr.stats.channel))
    plt.show()
    plt.close()

In [141]:
# MAIN

from eqcorrscan.utils.stacking import linstack
from eqcorrscan.core.match_filter import Party, Family
from obspy import read, Catalog
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
import math
from glob import glob
import os
from collections import Counter
import numpy as np

DETECT_DIR = "/home/gilbert_lab/cami_frs/borehole_data/energy_detector/detections_clean"
WF_DIR_ROOT_500Hz = "/home/gilbert_lab/cami_frs/borehole_data/sac_daily_nez_500Hz"
WF_DIR_ROOT_HAWK = "/home/gilbert_lab/cami_frs/hawk_data/sac_data_raw/"
WF_DIR_ROOT = "/home/gilbert_lab/cami_frs/sac_hourly_125Hz/"
SAMPLING_RATE = 125.0
ndt = 1/SAMPLING_RATE

party_dir = "/home/gilbert_lab/cami_frs/borehole_data/energy_detector/initial_templates_daily_old/"
out_dir = "/home/gilbert_lab/cami_frs/borehole_data/energy_detector/templates"

flist = glob(os.path.join(party_dir,"party_day202002*.tgz"))
days = [os.path.split(f)[1].split("_")[1].split(".")[0].split("day")[1] for f in flist]
days.sort()
days = days[14:]
total_count = 0
for daystr in days: 
    print("************ %s ************" % daystr)
    party = Party().read(os.path.join(party_dir,"party_day%s.tgz" % daystr))
    print(party)
    #party.decluster(trig_int=3.0)
    print("finished reading and declustering")
    if not party.families:
        print("no families...")
        continue
    count = 0
    for family in party:
        ndets = 0
        if not family:            
            continue
            
        # Determine good detections
        ndets = 0
        good_detections = []
        good_catalog = Catalog()
        for idet, d in enumerate(family.detections):
            detect_time = d.detect_time._get_datetime()
            is_ert_on = ert_surveys.loc[(ert_surveys['time_utc_start'] <= detect_time) & (ert_surveys['time_utc_end'] >= detect_time)].shape[0] > 0
            if not is_ert_on:
                ndets += 1
                good_detections.append(d)
                good_catalog.append(family.catalog[idet])
        if not good_detections:
            continue            
        new_fam = Family(template=family.template, detections=good_detections)
        
        # If family is good, save template and stack
        num_p_picks = len([p for p in new_fam.template.event.picks if p.phase_hint=="P"])
        if ndets > 2 and len(new_fam.template.st) > 4 and num_p_picks > 0:
            count +=1
            print("Family with %d detections" % ndets)
            full_template_st = get_full_template(new_fam.template)
            fname = os.path.join(out_dir, "%s_full_template.mseed" % new_fam.template.name)
            if os.path.exists(fname):
                os.remove(fname)
            new_fam.template.st.write(fname, format="MSEED") 
            full_template_st.write(fname, format="MSEED") 
            
            # stack
            fname = os.path.join(out_dir, "%s_stack.mseed" % new_fam.template.name)
            if os.path.exists(fname):
                os.remove(fname)
            stack = get_stack(new_fam)            
            stack.write(fname, format="MSEED") 
            
            # Family
            fname = os.path.join(out_dir, "%s_family.tgz" % new_fam.template.name)
            if os.path.exists(fname):
                os.remove(fname)
            new_fam.write(fname) 
            
    print("Number of good families: %d / %d" % (count, len(party.families)))
    total_count += count
    
print("total count = %d" % total_count)

************ 20200228 ************
Party of 19 Families.
finished reading and declustering
Number of good families: 0 / 19
************ 20200229 ************


KeyboardInterrupt: 

In [148]:
# Just get full template from tribe files

from eqcorrscan.core.match_filter import Tribe

flist = glob(os.path.join(party_dir,"tribe_day*.tgz"))
days = [os.path.split(f)[1].split("_")[1].split(".")[0].split("day")[1] for f in flist]
days.sort()
for daystr in days: 
    print("************ %s ************" % daystr)
    tribe = Tribe().read(os.path.join(party_dir,"tribe_day%s.tgz" % daystr))
    print(tribe)
    if not tribe.templates:
        print("no templates...")
        continue
    
    for template in tribe:        
        if not template:            
            continue                    
        fname = os.path.join(out_dir, "%s_full_template.mseed" % template.name)
        if os.path.exists(fname):
            continue
        num_p_picks = len([p for p in template.event.picks if p.phase_hint=="P"])
        has_magn = len(template.event.magnitudes) > 0
        if len(template.st) > 5 and (num_p_picks > 0 or has_magn):
            print(template.name)
            full_template_st = get_full_template(template)
            full_template_st.write(fname, format="MSEED") 
            print("Saving %s" % fname)
            fname = fname.replace("full_template.mseed", "template.tgz")
            template.write(fname)
            print("Saving %s" % fname)

************ 20200108 ************
Tribe of 160 templates
************ 20200109 ************
Tribe of 309 templates
************ 20200110 ************
Tribe of 56 templates
************ 20200122 ************
Tribe of 117 templates
2020_01_22t21_26_46
Saving /home/gilbert_lab/cami_frs/borehole_data/energy_detector/templates/2020_01_22t21_26_46_full_template.mseed
Saving /home/gilbert_lab/cami_frs/borehole_data/energy_detector/templates/2020_01_22t21_26_46_template.tgz
2020_01_22t21_35_42
Saving /home/gilbert_lab/cami_frs/borehole_data/energy_detector/templates/2020_01_22t21_35_42_full_template.mseed
Saving /home/gilbert_lab/cami_frs/borehole_data/energy_detector/templates/2020_01_22t21_35_42_template.tgz
2020_01_22t21_27_31
Saving /home/gilbert_lab/cami_frs/borehole_data/energy_detector/templates/2020_01_22t21_27_31_full_template.mseed
Saving /home/gilbert_lab/cami_frs/borehole_data/energy_detector/templates/2020_01_22t21_27_31_template.tgz
2020_01_22t21_34_19
Saving /home/gilbert_lab/c

Saving /home/gilbert_lab/cami_frs/borehole_data/energy_detector/templates/2020_01_24t16_08_21_full_template.mseed
Saving /home/gilbert_lab/cami_frs/borehole_data/energy_detector/templates/2020_01_24t16_08_21_template.tgz
************ 20200125 ************
Tribe of 1 templates
************ 20200126 ************
Tribe of 352 templates
2020_01_26t07_24_36
Saving /home/gilbert_lab/cami_frs/borehole_data/energy_detector/templates/2020_01_26t07_24_36_full_template.mseed
Saving /home/gilbert_lab/cami_frs/borehole_data/energy_detector/templates/2020_01_26t07_24_36_template.tgz
2020_01_26t07_55_28
Saving /home/gilbert_lab/cami_frs/borehole_data/energy_detector/templates/2020_01_26t07_55_28_full_template.mseed
Saving /home/gilbert_lab/cami_frs/borehole_data/energy_detector/templates/2020_01_26t07_55_28_template.tgz
2020_01_26t07_55_31
Saving /home/gilbert_lab/cami_frs/borehole_data/energy_detector/templates/2020_01_26t07_55_31_full_template.mseed
Saving /home/gilbert_lab/cami_frs/borehole_data/e

KeyboardInterrupt: 

In [149]:
days.index(daystr)

6

127